In [3]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

### Clean Data

In [2]:
nba = pd.ExcelFile('nba.xlsx')

In [3]:
nba.sheet_names

['Team Revenue - Total',
 'Team Revenue - Ticket',
 'Operating Income',
 'NBA Revenue',
 'Metro Populations',
 'Season Data',
 'Data Sources']

In [4]:
total = nba.parse('Team Revenue - Total')
total = total.rename({'Unnamed: 0': 'Team'}, axis=1)
total = total.set_index('Team')
total = total.unstack().reset_index()  
total = total.rename({'level_0': 'Year', 0: 'Total Revenue'}, axis=1)

In [5]:
ticket = nba.parse('Team Revenue - Ticket')
ticket = ticket.rename({'Unnamed: 0': 'Team'}, axis=1)
ticket = ticket.set_index('Team')
ticket = ticket.unstack().reset_index()  
ticket = ticket.rename({'level_0': 'Year', 0: 'Ticket Revenue'}, axis=1)

In [6]:
oi = nba.parse('Operating Income')
oi = oi.rename({'Unnamed: 0': 'Team'}, axis=1)
oi = oi.set_index('Team')
oi = oi.unstack().reset_index()  
oi = oi.rename({'level_0': 'Year', 0: 'Operating Income'}, axis=1)

In [7]:
total = pd.merge(total, ticket)
total = pd.merge(total, oi)

In [8]:
total

,Year,Team,Total Revenue,Ticket Revenue,Operating Income
0,2011,Atlanta Hawks,109,26.30,-15
1,2011,Boston Celtics,146,52.70,8
2,2011,Brooklyn Nets,89,23.30,-24
3,2011,Charlotte Hornets,101,19.50,-26
4,2011,Chicago Bulls,185,59.00,59
...,...,...,...,...,...
325,2021,Sacramento Kings,192,0.00,25
326,2021,San Antonio Spurs,205,2.37,39
327,2021,Toronto Raptors,194,1.62,2
328,2021,Utah Jazz,262,5.33,96


In [9]:
pops = nba.parse('Metro Populations')
pops = pops[['Team', 'Population, 2020', 'Population, 2010']]

In [10]:
total = pd.merge(total, pops)
total['Population'] = None
total.loc[total['Year'] <= 2020, 'Population'] = total.loc[total['Year'] <= 2020, 'Population, 2010']
total.loc[total['Year'] > 2020, 'Population'] = total.loc[total['Year'] > 2020, 'Population, 2020']

In [11]:
total = total.drop(['Population, 2020', 'Population, 2010'], axis=1)

In [12]:
seasons = nba.parse('Season Data')
seasons

,Team,Age,W,L,Attend.,Attend./G,Playoffs,Conference Semifinals,Conference Finals,Finals,Championship,Year
0,Orlando Magic,27.9,59,23,715901,17461,1.0,1.0,1,0,0.0,2010
1,Cleveland Cavaliers,28.0,61,21,843042,20562,1.0,1.0,0,0,0.0,2010
2,Utah Jazz,25.6,53,29,794512,19378,1.0,1.0,0,0,0.0,2010
3,San Antonio Spurs,28.7,50,32,741676,18090,1.0,1.0,0,0,0.0,2010
4,Atlanta Hawks,26.5,53,29,678375,16546,1.0,1.0,0,0,0.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
385,Detroit Pistons,23.6,23,59,663556,16184,0.0,0.0,0,0,0.0,2022
386,Orlando Magic,23.3,22,60,622881,15192,0.0,0.0,0,0,0.0,2022
387,Oklahoma City Thunder,22.4,24,58,595112,14515,0.0,0.0,0,0,0.0,2022
388,Houston Rockets,24.1,20,62,638977,15585,0.0,0.0,0,0,0.0,2022


In [13]:
data = pd.merge(total, seasons, how='left')

In [14]:
data['Future Revenue'] = data.groupby('Team')['Total Revenue'].shift(-1)
data['winPercent'] = data['W'] / (data['W'] + data['L'])
data['futureWinPercent'] =  data.groupby('Team')['winPercent'].shift(-1)

In [15]:
data = data[data['Year'] != 2021]

In [16]:
data = data.rename({'Total Revenue': 'TR'}, axis=1)

### Model #1 - Estimating Initial Population from Revenue

In [17]:
data['Population'] = data['Population'].astype(float)

In [18]:
results1 = smf.ols('TR ~ Population', data=data[data['Year']==2011]).fit()

In [19]:
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:                     TR   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     8.023
Date:                Mon, 05 Dec 2022   Prob (F-statistic):            0.00846
Time:                        14:48:34   Log-Likelihood:                -144.91
No. Observations:                  30   AIC:                             293.8
Df Residuals:                      28   BIC:                             296.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    114.2717      8.484     13.468      0.0

In [20]:
def rev2pop(x):
    return (x - results1.params[0]) / results1.params[1]

def pop2rev(x):
    return x * results1.params[1] + results1.params[0]

In [21]:
rev2pop(pop2rev(1e6) * 4)

106547254.5426164

### Model #2 - Updating Revenue

In [22]:
data['FR'] = data['Future Revenue']

In [23]:
def logit(x):
    return np.log(x / (1-x))

def inverse_logit(x):
    return np.exp(x) / (1 + np.exp(x))

def indicator_clip(x, epsilon=1e-5):
    x = np.where(x == 1, x-epsilon, x)
    x = np.where(x == 0, epsilon, x)
    return x

In [24]:
data['transformedWinPercent'] = logit(data['winPercent'])

In [25]:
results2 = smf.ols('FR ~ Championship + transformedWinPercent + TR + Population', data=data).fit()

In [26]:
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                     FR   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     320.1
Date:                Mon, 05 Dec 2022   Prob (F-statistic):          8.92e-106
Time:                        14:48:36   Log-Likelihood:                -1459.3
No. Observations:                 299   AIC:                             2929.
Df Residuals:                     294   BIC:                             2947.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                37.35

In [27]:
def update_revenue(champ, twp, tr, pop):
    return results2.params[0] +\
        results2.params[1] * champ +\
        results2.params[2] * twp +\
        results2.params[3] * tr +\
        results2.params[4] * pop

### Model #2 - Updating Win Percentage

In [28]:
def elo(wp, te, p, beta=100):
    return p * wp + (1-p) / (1 + np.power(10, -te / beta))

### Historical Data

In [29]:
data['TR'].max(), data['Population'].max()

(474, 19567410.0)

In [30]:
data['TR'].min(), data['Population'].min()

(78, 1124197.0)

In [31]:
data.groupby('Year')['TR'].max() / data.groupby('Year')['TR'].min()

Year
2011    2.741573
2012    3.115385
2013    2.706422
2014    2.663636
2015    2.475806
2016    2.685714
2017    2.379888
2018    2.171569
2019    2.107143
2020    2.267943
Name: TR, dtype: float64

### Running Simulations - Constant Tax Rate

In [32]:
nteams = 2
nsims = 100
nseasons = 30
ngames = 82

In [4]:
ps = np.array([0, 0.01, 0.05, 0.1, 0.25, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975, 0.99, 1])

In [5]:
step2 = 0.5
ratios = np.arange(1, 4 + step2, step2)

In [6]:
step3 = 0.25
taxes = np.arange(0, 5 + step3, step3)

In [7]:
step4 = 10
caps = np.arange(100, 800 + step4, step4)

In [37]:
def luxury_tax(spending, rate, cap):
    return rate * np.maximum(0, spending-cap)

def calculate_spending(capital, rate, cap):
    return np.where(capital > cap, (capital + rate * cap) / (1 + rate), capital)

In [38]:
base_pop = 1e6
base_capital = pop2rev(base_pop)

In [8]:
len(ps) * len(ratios) * len(taxes) * len(caps)

166992

In [42]:
len(ps), len(ratios), len(taxes), len(caps)

(16, 7, 21, 71)

In [46]:
results_shape = (len(ratios), len(ps), len(taxes), len(caps), nteams, nsims, nseasons)
capital = np.zeros(results_shape)
pop = np.zeros(results_shape)
spending = np.zeros(results_shape)
probs = np.zeros(results_shape)
wins = np.zeros(results_shape)
champs = np.zeros(results_shape)
tp = np.zeros((len(ratios), len(ps), len(taxes), len(caps), nsims))

In [47]:
for r in tqdm(range(len(ratios))):
    
    init_capital = np.array([base_capital, ratios[r] * base_capital])
    init_pop = np.array([base_pop, rev2pop(ratios[r] * base_capital)])
    
    for p in tqdm(range(len(ps))):
        for t in range(len(taxes)):
            for c in range(len(caps)):

                # set initial capital and win probabilities
                capital[r, p, t, c, :, :, 0] = np.repeat(init_capital[:, np.newaxis], nsims, axis=1)
                probs[r, p, t, c, :, :, 0] = 0.5

                for s in range(nseasons):

                    # set population as constant
                    pop[r, p, t, c, :, :, s] = np.repeat(init_pop[:, np.newaxis], nsims, axis=1)

                    # calculate spending
                    spending[r, p, t, c, :, :, s]= calculate_spending(capital[r, p, t, c, :, :, s],
                                                                      taxes[t],
                                                                      caps[c])
                    
                    #calculate taxes
                    tp[r, p, t, c, :] += (capital[r, p, t, c, 0, :, s] - spending[r, p, t, c, 0, :, s])
                    tp[r, p, t, c, :] += (capital[r, p, t, c, 1, :, s] - spending[r, p, t, c, 1, :, s])

                    # calculate regular season wins
                    wins[r, p, t, c, 0, :, s] = np.random.binomial(ngames, probs[r, p, t, c, 0, :, s])
                    wins[r, p, t, c, 1, :, s] = ngames - wins[r, p, t, c, 0, :, s]

                    # calculate championship winner
                    winners = np.random.binomial(1, wins[r, p, t, c, :, :, s].max(0) / ngames)
                    winners = np.repeat(winners[np.newaxis, :], nteams, axis=0)
                    favorites = wins[r, p, t, c, :, :, s] == wins[r, p, t, c, :, :, s].max(0)
                    champs[r, p, t, c, :, :, s] = np.where(favorites, winners, 1-winners)

                    if s < nseasons - 1:
                        update = update_revenue(champs[r, p, t, c, :, :, s],
                                                logit(indicator_clip(wins[r, p, t, c, :, :, s] / ngames)),
                                                capital[r, p, t, c, :, :, s],
                                                pop[r, p, t, c, :, :, s])
                        capital[r, p, t, c, :, :, s+1] = update
                        te = (spending[r, p, t, c, :, :, s] - spending[r, p, t, c, :, :, s].mean(0))
                        probs[r, p, t, c, :, :, s+1] = elo(probs[r, p, t, c, :, :, s], te, p=ps[p])

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [48]:
parity = champs[:, :, :, :, 0, :, :].sum(-1).mean(-1)

In [54]:
#effect of parameters on baseline parity
fig, axs = plt.subplots(nrows=1, ncols=1)
sns.set(rc={'figure.figsize':(10, 10)})
ax = sns.heatmap(parity.min(-1).min(-1), annot=True, ax=axs)
ax.set(xlabel='q', ylabel='Ratio')
ax.set_xticklabels(np.round(ps, decimals=3))
ax.set_yticklabels(ratios)
plt.savefig('baseline_parity.png')
plt.close(fig)

In [126]:
sns.set(rc={'figure.figsize':(len(ps)*len(caps)*(2/3), len(ratios)*len(taxes)*(2/3))})

In [56]:
fig, axs = plt.subplots(nrows=len(ratios), ncols=len(ps))

for r in range(len(ratios)):
    for p in range(len(ps)):
        temp_mean = parity[r, p, :, :].mean()
        temp_std = parity[r, p, :, :].std()
        temp_plot = (parity[r, p, :, :] - temp_mean) / temp_std
        ax = sns.heatmap(temp_plot, annot=False, ax=axs[r, p])
        ax.set(xlabel='Caps', ylabel='Taxes', title=f'Ratio: {ratios[r]}, q: {np.round(ps[p], decimals=3)}, mean: {np.round(temp_mean, decimals=1)}, std: {np.round(temp_std, decimals=1)}')
        ax.set_xticklabels(caps)
        ax.set_yticklabels(taxes)
        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(f'parity/parity_{ratios[r]}_{np.round(ps[p], decimals=3)}.png',
                    bbox_inches=extent.expanded(1.4, 1.2))
plt.close(fig)

In [57]:
total_tp = tp.mean(-1)

In [60]:
fig, axs = plt.subplots(nrows=len(ratios), ncols=len(ps))

for r in range(len(ratios)):
    for p in range(len(ps)):
        temp_mean = total_tp[r, p, :, :].mean()
        temp_std = total_tp[r, p, :, :].std()
        temp_plot = (total_tp[r, p, :, :] - temp_mean) / temp_std
        ax = sns.heatmap(temp_plot, annot=False, ax=axs[r, p])
        ax.set(xlabel='Caps', ylabel='Taxes', title=f'Ratio: {ratios[r]}, q: {np.round(ps[p], decimals=3)}, mean: {np.round(temp_mean, decimals=1)}, std: {np.round(temp_std, decimals=1)}')
        ax.set_xticklabels(caps)
        ax.set_yticklabels(taxes)
        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(f'taxes/tp_{ratios[r]}_{np.round(ps[p], decimals=3)}.png',
                    bbox_inches=extent.expanded(1.4, 1.2))
plt.close(fig)

In [61]:
baseline = parity.min(-1).min(-1)
expanded_baseline = np.repeat(baseline[:, :, np.newaxis], len(taxes), axis=-1)
expanded_baseline = np.repeat(expanded_baseline[:, :, :, np.newaxis], len(caps), axis=-1)

In [62]:
obj = (parity - expanded_baseline) / total_tp

/Users/matthewsmith/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/Users/matthewsmith/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [63]:
obj = obj[1:, :-1, 1:, :]
obj = np.where(obj == np.inf, 0, obj)
obj = np.where(obj == -np.inf, 0, obj)
obj = np.nan_to_num(obj)

In [67]:
fig, axs = plt.subplots(nrows=len(ratios)-1, ncols=len(ps)-1)

for r in range(1, len(ratios)):
    for p in range(len(ps)-1):  
        temp_mean = obj[r-1, p, :, :].mean()
        temp_std = obj[r-1, p, :, :].std()
        temp_plot = (obj[r-1, p, :, :] - temp_mean) / temp_std
        ax = sns.heatmap(temp_plot, annot=False, ax=axs[r-1, p])
        ax.set(xlabel='Caps', ylabel='Taxes', title=f'Ratio: {ratios[r]}, q: {np.round(ps[p], decimals=3)}, mean: {np.round(temp_mean, decimals=1)}, std: {np.round(temp_std, decimals=1)}')
        ax.set_xticklabels(caps)
        ax.set_yticklabels(taxes[1:])
        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(f'objective/obj_{ratios[r]}_{np.round(ps[p], decimals=3)}.png',
                    bbox_inches=extent.expanded(1.4, 1.2))
plt.close(fig)

In [146]:
sns.set(rc={'figure.figsize':(20, 20)})

In [143]:
capital[:, :, 0, 0, 0, :, :].max(), capital[:, :, 0, 0, 0, :, :].min()

(265.99267075815396, -100.15747405570632)

In [147]:
# how revenue changes for small market team
for r in range(len(ratios)):
    for p in range(len(ps)):
        
        fig = plt.figure()

        for i in range(nsims):
            plt.plot(range(nseasons), capital[r, p, 0, 0, 0, i, :])

        plt.title(f'Ratio: {ratios[r]}, q: {np.round(ps[p], decimals=3)}')
        plt.ylabel('Revenue - Small Market Team')
        plt.xlabel('Season')
        plt.ylim([-150, 300])
            
        fig.savefig(f'capital_small/rev_{ratios[r]}_{np.round(ps[p], decimals=3)}.png')
            
        plt.close(fig)

In [148]:
capital[:, :, 0, 0, 1, :, :].max(), capital[:, :, 0, 0, 1, :, :].min()

(1308.7761249674918, 117.61464801446984)

In [149]:
# how revenue changes for large market team
for r in range(len(ratios)):
    for p in range(len(ps)):
        
        fig = plt.figure()

        for i in range(nsims):
            plt.plot(range(nseasons), capital[r, p, 0, 0, 1, i, :])

        plt.title(f'Ratio: {ratios[r]}, q: {np.round(ps[p], decimals=3)}')
        plt.ylabel('Revenue - Large Market Team')
        plt.xlabel('Season')
        plt.ylim([50, 1350])
            
        fig.savefig(f'capital_large/rev_{ratios[r]}_{np.round(ps[p], decimals=3)}.png')
            
        plt.close(fig)

In [68]:
rev2pop(4 * base_capital)

106547254.5426164

In [108]:
expanded_caps = np.repeat(caps[np.newaxis, :], len(taxes)-1, axis=0)
expanded_caps = np.repeat(expanded_caps[np.newaxis, :, :], len(ps)-1, axis=0)
expanded_caps = np.repeat(expanded_caps[np.newaxis, :, :, :], len(ratios)-1, axis=0)

In [109]:
expanded_max = np.max(obj, axis=(2, 3))
expanded_max = np.repeat(expanded_max[:, :, np.newaxis], len(taxes)-1, axis=-1)
expanded_max = np.repeat(expanded_max[:, :, :, np.newaxis], len(caps), axis=-1)

In [112]:
optimal_caps = expanded_caps[obj == expanded_max].reshape((len(ratios)-1, len(ps)-1))

In [152]:
sns.set(rc={'figure.figsize':(20, 10)})

fig, axs = plt.subplots(nrows=1, ncols=1)
ax = sns.heatmap(optimal_caps, annot=True, ax=axs)
ax.set_xticklabels(ps[:-1])
ax.set_yticklabels(ratios[1:])
ax.set(xlabel='q', ylabel='Ratio')
plt.savefig('optimal_cap.png')
plt.close(fig)